In [7]:
from bs4 import BeautifulSoup

# Collect Data

### Scrape Annenberg Landing Page

In [2]:
# mv /Users/spangher/Downloads/one_year_annenberg_media.html .

In [6]:
landing_page_html = open('../data/one_year_annenberg_media.html').read()

In [8]:
soup = BeautifulSoup(landing_page_html)

In [11]:
all_list_items = soup.find_all('div', attrs={'class': 'list-item'})

In [12]:
len(all_list_items)

40

In [15]:
import asyncio
from playwright.async_api import async_playwright

playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page()
        
# To run this in Jupyter, use:
url = "https://www.uscannenbergmedia.com/allnews/"
await page.goto(url)

<Response url='https://www.uscannenbergmedia.com/allnews/' request=<Request url='https://www.uscannenbergmedia.com/allnews/' method='GET'>>

In [16]:
n_scrolls = 5  # Replace with however many scrolls you want
# page_content = await scroll_and_click_more(url, n_scrolls)

# To display the page content in a notebook
# print(page_content[:1000])  # Display the first 1000 characters

In [20]:
import time
from tqdm.auto import tqdm
for _ in tqdm(range(100)):
    # Scroll to the bottom of the page
    await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
    await asyncio.sleep(1)  # Give the page time to load

    try:
        await page.click("div.see-more")
    except:
        print("See More button not found or no more content to load.")
        break
    
    # Optional delay to avoid rate limiting or too-fast scrolling
    time.sleep(2)

  0%|          | 0/100 [00:00<?, ?it/s]

In [21]:
html = await page.content()

In [25]:
with open('../data/batch_collection_annenberg_media.html', 'w') as f:
    f.write(html)

### Scrape article pages

In [22]:
soup = BeautifulSoup(html)

In [23]:
all_list_items = soup.find_all('div', attrs={'class': 'list-item'})

In [39]:
all_urls = list(map(lambda x: x.find('a')['href'], all_list_items))

In [40]:
homepage_domain = 'https://www.uscannenbergmedia.com'

In [41]:
# await page.goto(url)

In [32]:
import requests

In [43]:
all_article_pages = []
for url in tqdm(all_urls):
    r = requests.get(f'{homepage_domain}/{url}')
    all_article_pages.append(r.text)

  0%|          | 0/4240 [00:00<?, ?it/s]

In [47]:
ls ../data/

batch_collection_annenberg_media.html  one_year_annenberg_media.html


In [48]:
import pickle
with open('../data/batch_collection_annenberg_html.pkl', 'wb') as f:
    pickle.dump( all_article_pages, f)

In [50]:
all_articles_parsed = list(map(lambda x: BeautifulSoup(x).get_text(' '), all_article_pages))

In [51]:
import pandas as pd 

In [54]:
article_df = pd.DataFrame([all_urls, all_articles_parsed], index=['url', 'article_text']).T

In [56]:
article_df.to_csv('../data/batch_article_text.csv')

In [57]:
article_df

,url,article_text
0,/2024/10/08/dream-big-says-ezra-frech-a-two-ti...,"‘Dream big’ says Ezra Frech, a two-time Paraly..."
1,/2024/10/08/the-big-ten-breakdown-vol-3/,The Big Ten Breakdown Vol. 3: – Annenberg Medi...
2,/2024/10/08/review-linkin-park-marks-their-com...,Review: Linkin Park marks their comeback on a ...
3,/2024/10/07/call-her-president/,Call Her President? – Annenberg Media \n \n ...
4,/2024/10/07/usc-jewish-student-organizations-h...,USC Jewish student organizations host vigil fo...
...,...,...
4235,/2022/09/30/where-yoga-and-art-meet-how-one-in...,Where yoga and art meet: how one instructor br...
4236,/2022/09/30/grgurevic-mckenney-lead-usc-to-a-c...,"Grgurevic, McKenney lead USC to a comeback vic..."
4237,/2022/09/30/is-our-livelihood-not-important-re...,“Is our livelihood not important?”: Residents ...
4238,/2022/09/30/newsom-signs-law-allowing-californ...,Newsom signs law allowing California farmworke...


In [44]:
# print(BeautifulSoup(r.text).get_text(' '))